In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import csv
import time

# Initialize the WebDriver
driver = webdriver.Chrome()

# Open the target website
driver.get('https://www.rootdata.com/Market')

driver.maximize_window()

# 设置等待时间
wait = WebDriverWait(driver, 3)

# 关闭"立即体验"和弹窗
try:
    button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div/div[3]/div/div/div[2]/div[2]/button"))
    )
    button.click()
    div1 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div/div[1]/div[4]/div"))
    )
    div1.click()
except Exception as e:
    print(f"step 1: An unexpected error occurred: {e}")

# 初始化一个空列表来存储数据
top_10_data = []

def get_project_data():
    projects = driver.find_elements(By.XPATH, '//tbody/tr')
    for project in projects:
        try:
            rank = project.find_element(By.XPATH, './td[2]').text.strip()
            name_element = project.find_element(By.XPATH, './td[3]//div[@class="d-flex name"]//a')
            name = name_element.text.strip()
            price = project.find_element(By.XPATH, './td[4]').text.strip()
            market_cap = project.find_element(By.XPATH, './td[8]').text.strip()
            fdv = project.find_element(By.XPATH, './td[9]').text.strip()
            link = name_element.get_attribute('href')
            data.append({
                'Rank': rank,
                'Name': name,
                'Price': price,
                'Market Cap': market_cap,
                'FDV': fdv,
                'Link': link
            })
        except Exception as e:
            print(f"Error in parsing project: {e}")

try:
    table = wait.until(EC.presence_of_element_located((By.ID, '__BVID__143')))
    for _ in range(3):  # 假设每次滚动可以加载10条数据，需要滚动3次
        rows = table.find_elements(By.TAG_NAME, 'tr')
        # 从第2行开始（第1行是表头），遍历每一行
        for row in rows[1:]:
            # 获取每个单元格的数据
            cells = row.find_elements(By.TAG_NAME, 'td')
            # 提取所需数据
            rank = cells[1].text.strip()
            name_element = cells[3].find_element(By.XPATH, './div[@class="d-flex name"]//a')    # 不对
            name = name_element.text.strip()
            # name = cells[3].find_element(By.CLASS_NAME, 'd-flex name').text.strip()
            price = cells[4].text.strip()
            change_24h = cells[5].text.strip()
            change_7d = cells[6].text.strip()
            volume_24h = cells[7].text.strip()
            market_cap = cells[8].text.strip()
            fdv = cells[9].text.strip()
            ecosystem = cells[10].text.strip()
            exchange = cells[11].text.strip()
            url = name_element.get_attribute('href')
            # url = cells[3].find_element(By.CLASS_NAME, 'd-flex name').find_element(By.TAG_NAME, 'a').get_attribute('href')
            
            # 将数据添加到列表中
            top_10_data.append({
                'rank': rank,
                'name': name,
                'price': price,
                'change_24h': change_24h,
                'change_7d': change_7d,
                'volume_24h': volume_24h,
                'market_cap': market_cap,
                'fdv': fdv,
                'ecosystem': ecosystem,
                'exchange': exchange,
                'url': url
            })
        
        # 如果已经获取到**10**条数据，则停止滚动
        if len(top_10_data) >= 10:
            break
        
        # 滚动页面
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # 等待新数据加载
        wait.until(EC.presence_of_element_located((By.TAG_NAME, 'tr')))
except Exception as e:
    print(f"step 2: An unexpected error occurred: {e}")
finally:
    driver.quit()



# 写入CSV文件
with open('coin.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Rank', 'Name', 'Price', 'Change 24h', 'Change 7d', 'Volume 24h', 'Market Cap', 'FDV', 'Ecosystem', 'Exchange', 'URL'])
    for data in top_10_data:
        writer.writerow([
            data['rank'],
            data['name'],
            data['price'],
            data['change_24h'],
            data['change_7d'],
            data['volume_24h'],
            data['market_cap'],
            data['fdv'],
            data['ecosystem'],
            data['exchange'],
            data['url']
        ])

def navigate_pages():
    page_number = 1
    while True:
        print(f"Currently on page {page_number}")
        get_project_data()
        try:
            # 檢查是否存在下一頁按鈕
            wait = WebDriverWait(driver, 10)
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn-next')))
            next_button.click()
            time.sleep(5)  # 等待頁面加載新數據
            page_number += 1
        except Exception as e:
            print(f"No more pages or an error occurred: {e}")
            break

# 開始抓取數據
navigate_pages()

# 關閉 WebDriver
driver.quit()

# 將數據導出到 CSV 文件
df = pd.DataFrame(data)
df.to_csv('market_projects.csv', index=False)

print("Data has been exported to market_projects.csv")



step 2: An unexpected error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div[@class="d-flex name"]//a"}
  (Session info: chrome=128.0.6613.139); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DBF19412+29090]
	(No symbol) [0x00007FF6DBE8E239]
	(No symbol) [0x00007FF6DBD4B1DA]
	(No symbol) [0x00007FF6DBD9EFE7]
	(No symbol) [0x00007FF6DBD9F23C]
	(No symbol) [0x00007FF6DBD927EC]
	(No symbol) [0x00007FF6DBDC672F]
	(No symbol) [0x00007FF6DBD926C6]
	(No symbol) [0x00007FF6DBDC6900]
	(No symbol) [0x00007FF6DBDE65A2]
	(No symbol) [0x00007FF6DBDC6493]
	(No symbol) [0x00007FF6DBD909D1]
	(No symbol) [0x00007FF6DBD91B31]
	GetHandleVerifier [0x00007FF6DC23871D+3302573]
	GetHandleVerifier [0x00007FF6DC284243+3612627]
	GetHandleVerifier [0x00007FF6DC27A417+3572135]
	GetHandleVerifier [0x00007FF6DBFD5EB6+801862]
	(No

MaxRetryError: HTTPConnectionPool(host='localhost', port=63646): Max retries exceeded with url: /session/51c33ba2a543963d65482904a95fcd5d/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EBEB869E70>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))